In [1]:
wd <- dirname(getwd())
source(paste0(wd,"/mission_control/treasure_map.R"))
source(paste0(wd,"/mission_control/after_burn_help.R"))

library(tidyverse)
library(data.table)

args <- list("AdjTPM")

Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”
Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.1
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ─────────────────────────────────────────────────────────────────────────

#### 0 - Bring all the Data together

- Read output from each pipeline

In [2]:
setwd(TMP_DIR)

In [3]:
clinical       <- fread( "clinical_ready.csv" )
summary        <- fread( "summary_ready.csv" )
sigs           <- fread( "sigs_ready.csv" )
cibersort_lm22 <- fread( paste0("cibersort_", args[1],"_LM22_ready.csv") )
cibersort_tr4  <- fread( paste0("cibersort_", args[1],"_TR4_ready.csv") )
drivers        <- fread( "driver_ready.csv" )
hlas           <- fread( "hla_ready.csv" )
isofox         <- fread( paste0("isofox_", args[1], "_ready.csv") )
somatic        <- fread( "somatic_ready.csv" )
cnv            <- fread( "cnv_ready.csv" )
cnv_gene       <- fread( "cnv_gene_ready.csv" )
neoepitope     <- fread( "neoepitope_ready.csv" )
sv             <- fread( "sv_ready.csv" )
lilac          <- fread( "lilac_ready.csv")

#### Add heterogeneity proxy with clonal pct

In [4]:
somatic <- somatic %>% mutate(somatic_clonal_pct = somatic_TMB_clonal/somatic_TMB)
somatic$somatic_TMB_exome <- apply( somatic %>% select(contains(".gene")), 1, sum)

#### Log-Transforms
- Somatic, CNV, Neoepitope, Svs

In [5]:
somatic <- somatic %>% mutate_at(vars(-sampleId, -somatic_clonal_pct), ~(log(.+1) %>% as.vector))
cnv <- cnv %>% mutate_at( vars(-sampleId), ~(log(.+1) %>% as.vector))
cnv_gene <- cnv_gene %>% mutate_at(vars(-contains("_loh_"), -sampleId), ~(log(.+1) %>% as.vector))
neoepitope <- neoepitope %>% mutate_at(vars(-sampleId), ~(log(.+1) %>% as.vector))
sv <- sv %>% mutate_at(vars(-sampleId), ~(log(.+1) %>% as.vector))

Warning message in log(. + 1):
“NaNs produced”Warning message in log(. + 1):
“NaNs produced”Warning message in log(. + 1):
“NaNs produced”Warning message in log(. + 1):
“NaNs produced”

#### All together! 

In [6]:
all <- (
    clinical
        %>% left_join(hlas, by='sampleId')
        %>% left_join(cibersort_lm22, by='sampleId')
        %>% left_join(cibersort_tr4, by='sampleId')
        %>% left_join(cnv, by='sampleId')
        %>% left_join(cnv_gene, by='sampleId')
        %>% left_join(drivers, by='sampleId')
        %>% left_join(isofox, by='sampleId')
        %>% left_join(neoepitope, by='sampleId')
        %>% left_join(sigs, by='sampleId')
        %>% left_join(somatic, by='sampleId')
        %>% left_join(summary, by='sampleId') 
        %>% left_join(sv, by='sampleId')   
        %>% left_join(lilac, by='ID_meta_hmfSampleId')
    )

In [7]:
#all %>% select(contains("cluster"))

### 1 - Add PC features

In [8]:
#all <- 
#    (all 
#         %>% left_join( give_me_pcs(somatic, "somatic_"), by = "sampleId")
#         %>% left_join( give_me_pcs(somatic, "somatic.gene_"), by = "sampleId")
#         %>% left_join( give_me_pcs(isofox, "isofox"), by = "sampleId")
#         #%>% left_join( give_me_pcs(cnv_gene, "cnv.region_cn_"), by = "sampleId")
#)

### 2 - Send it!

In [9]:
fwrite( all, file=paste0( CLN_DIR, "signals_base.csv") )